In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy
import os
import seaborn
pd.set_option("display.max_columns",999)

In [83]:
chromedriver = "/Users/brian_newborn/Downloads/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
#2010-2017 Batting Stats, Advanced
url="https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=100&type=1&season=2017&month=0&season1=2010&ind=1&team=0&rost=0&age=0&filter=&players=0"
driver.get(url)

#Need to Save Column Names
col_names_list = []
master_players={} 

In [84]:
def next_pg():
    next_section_sp =''
    #Then, try next page button - we want all names within a section
    try:
        next_pg = driver.find_element_by_xpath('//a[@title="Next Page"]')
        next_pg.send_keys(u'\ue007')
        
    except: #End of Section, go to next section
        print("End of Section")
        next_section= driver.find_elements_by_xpath\
        ('.//div[@class = "rtsLevel rtsLevel1"]//li[@class="rtsLI"]//a[@class = "rtsLink rtsAfter"]')
        next_section_sp = next_section[3]
        print(f"Next Section: {next_section_sp.text}")
        next_section_sp.send_keys(u'\ue007')
    return next_section_sp

def one_section_only():
    try:
        next_pg = driver.find_element_by_xpath('//a[@title="Next Page"]')
        next_pg.send_keys(u'\ue007')
        print("Next Page")
        keep_running=True
    except:
        print("End of Section")
        keep_running =False
    return keep_running

def col_headers():
    #Grab Headers - if they arent in list
    identifier_header = "rgHeader"
    
    col_names = driver.find_elements_by_class_name(identifier_header)
    curr_page = driver.find_element_by_xpath\
    ('.//div[@class = "rgWrap rgNumPart"]//a[@class="rgCurrentPage"]').text
    for n in col_names[4:]:
        col_names_list.append(n.text)
        print(n.text)
    return col_names_list



def scrape_players():
    #Grab Headers - if they arent in list
    identifier_header = "rgHeader"
    
    col_names = driver.find_elements_by_class_name(identifier_header)
    curr_page = driver.find_element_by_xpath\
    ('.//div[@class = "rgWrap rgNumPart"]//a[@class="rgCurrentPage"]').text
    
    #Grab Players - 2 Types of HTML rows
    identifier1,identifier2 = "rgRow","rgAltRow"

    players = driver.find_elements_by_class_name(identifier2)
    players2 = driver.find_elements_by_class_name(identifier1)
    players = players+players2
    
    for player in players:
#         player = player.find_elements_by_class_name("grid_line_regular")
#         print(player)
        player = player.find_elements_by_tag_name('td') #td elements-need each element       
#         print(player)
        name =  (player[2].text)+' '+(player[1].text) #name + year
#         print(f"Name: {name}")
        for td in player:
            if name in master_players: #if player already exists, append item to 
                master_players[name].append(td.text)

            else:
                master_players[name]=[td.text]


In [85]:
columns = col_headers()

PA
BB%
K%
BB/K
AVG
OBP
SLG
OPS
ISO
Spd
BABIP
UBR
wGDP
wSB
wRC
wRAA
wOBA
wRC+


In [86]:
keep_running = True #flag to keep loop going
scrape_players()
while keep_running == True:
        keep_running = one_section_only()
        scrape_players()

Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page
Next Page


In [88]:
df = pd.DataFrame.from_dict(data=master_players,orient="index")

In [87]:
columns

['PA',
 'BB%',
 'K%',
 'BB/K',
 'AVG',
 'OBP',
 'SLG',
 'OPS',
 'ISO',
 'Spd',
 'BABIP',
 'UBR',
 'wGDP',
 'wSB',
 'wRC',
 'wRAA',
 'wOBA',
 'wRC+']

In [90]:
df.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43
Josh Vitters 2012,3559,2012,Josh Vitters,Cubs,109,6.4 %,30.3 %,0.21,.121,.193,.202,.395,.081,4.4,.154,-1.2,-1.0,0.3,1,-11.5,.183,5,3559,2012,Josh Vitters,Cubs,109,6.4 %,30.3 %,0.21,.121,.193,.202,.395,.081,4.4,.154,-1.2,-1.0,0.3,1,-11.5,.183,5
Pete Kozma 2015,3561,2015,Pete Kozma,Cardinals,111,9.0 %,18.9 %,0.48,.152,.236,.152,.388,.000,5.3,.192,1.6,0.7,0.2,1,-11.8,.180,8,3561,2015,Pete Kozma,Cardinals,111,9.0 %,18.9 %,0.48,.152,.236,.152,.388,.000,5.3,.192,1.6,0.7,0.2,1,-11.8,.180,8
Leury Garcia 2014,3563,2014,Leury Garcia,White Sox,155,3.2 %,31.0 %,0.10,.166,.192,.207,.399,.041,6.8,.237,1.2,-0.7,1.7,1,-15.8,.177,4,3563,2014,Leury Garcia,White Sox,155,3.2 %,31.0 %,0.10,.166,.192,.207,.399,.041,6.8,.237,1.2,-0.7,1.7,1,-15.8,.177,4
Luis Cruz 2013,3565,2013,Luis Cruz,- - -,187,3.2 %,17.6 %,0.18,.145,.190,.179,.369,.035,4.2,.171,0.6,-0.4,0.1,-1,-21.2,.169,-1,3565,2013,Luis Cruz,- - -,187,3.2 %,17.6 %,0.18,.145,.190,.179,.369,.035,4.2,.171,0.6,-0.4,0.1,-1,-21.2,.169,-1
Casper Wells 2013,3567,2013,Casper Wells,- - -,102,6.9 %,30.4 %,0.23,.126,.186,.147,.334,.021,2.2,.188,-0.7,0.1,-0.4,-1,-12.3,.159,-10,3567,2013,Casper Wells,- - -,102,6.9 %,30.4 %,0.23,.126,.186,.147,.334,.021,2.2,.188,-0.7,0.1,-0.4,-1,-12.3,.159,-10


In [98]:
# df = df.drop(labels=range(22,44,1),axis=1)
# df = df.drop(labels=[0,1,2,3],axis=1)



In [96]:
# len(col_names_list) #18
col_names_list

['PA',
 'BB%',
 'K%',
 'BB/K',
 'AVG',
 'OBP',
 'SLG',
 'OPS',
 'ISO',
 'Spd',
 'BABIP',
 'UBR',
 'wGDP',
 'wSB',
 'wRC',
 'wRAA',
 'wOBA',
 'wRC+']

In [97]:
df.head()

,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,43
Miguel Cabrera 2013,652,13.8 %,14.4 %,0.96,.348,.442,.636,1.078,.288,2.8,.356,-4.9,-1.0,-0.1,144,72.1,.455,193,None
Josh Hamilton 2010,571,7.5 %,16.6 %,0.45,.359,.411,.633,1.044,.274,4.7,.390,3.3,0.9,0.7,122,56.8,.445,175,None
Troy Tulowitzki 2014,375,13.3 %,15.2 %,0.88,.340,.432,.603,1.035,.263,3.1,.355,0.0,0.8,-0.6,79,38.5,.444,170,None
Hanley Ramirez 2013,336,8.0 %,15.5 %,0.52,.345,.402,.638,1.040,.293,6.0,.363,-2.5,0.8,0.9,71,33.8,.442,191,None
Joey Votto 2012,475,19.8 %,17.9 %,1.11,.337,.474,.567,1.041,.230,2.3,.404,-2.3,1.1,-0.9,101,47.0,.438,178,None


In [99]:
df.columns = col_names_list
# len(df.columns)

In [37]:
df.loc["Josh Hamilton 2010"]

4        571
5      7.5 %
6     16.6 %
7       0.45
8       .411
9       .633
10     1.044
11       4.7
12      .390
13       0.9
14       0.7
15      56.8
16      .445
17       175
Name: Josh Hamilton 2010, dtype: object

In [100]:
import pickle

with open('df_advanced.pkl', 'wb') as picklefile:
    pickle.dump(df, picklefile)